Importimine

In [10]:
import zipfile
import os
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Model
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display

%matplotlib inline

Kopsupõletikku tuvastamine

In [11]:

# Path to the ZIP file
zip_path = "archive.zip"
extract_path = "extracted_photos"

# Extract the ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_file:
    zip_file.extractall(extract_path)


zip_path = "archive (1).zip"
#Extract the ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_file:
    zip_file.extractall(extract_path)

zip_path = "archive (2).zip"
# Extract the ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_file:
    zip_file.extractall(extract_path)



Kood, mis eraldab kopsupõletiku bakteriaalseks ja viiruslikuks

In [12]:
import os
import shutil

#Treening and testiandmed
base_dirs = ['extracted_photos/Train', 'extracted_photos/Test', 
             'extracted_photos/chest_xray/chest_xray/train', 'extracted_photos/chest_xray/chest_xray/test',
            'extracted_photos/chest_xray/train', 'extracted_photos/chest_xray/test']


bacteria_folder = 'PNEUMONIA_BACTERIA'
virus_folder = 'PNEUMONIA_VIRUS'

for base_dir in base_dirs:
    pneumonia_path = os.path.join(base_dir, 'PNEUMONIA')
    
    if os.path.exists(pneumonia_path):
        
        bacteria_path = os.path.join(base_dir, bacteria_folder)
        virus_path = os.path.join(base_dir, virus_folder)
        os.makedirs(bacteria_path, exist_ok=True)
        os.makedirs(virus_path, exist_ok=True)
        
        for filename in os.listdir(pneumonia_path):
            if 'bacteria' in filename:
                shutil.move(os.path.join(pneumonia_path, filename), os.path.join(bacteria_path, filename))
            elif 'virus' in filename:
                shutil.move(os.path.join(pneumonia_path, filename), os.path.join(virus_path, filename))
        
        shutil.rmtree(pneumonia_path)
    else:
        print("Error")


Error
Error


In [13]:
import shutil

def create_symlinks(src_dirs, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for src_dir in src_dirs:
        for root, dirs, files in os.walk(src_dir):
            for file in files:
                src_file = os.path.join(root, file)
                relative_path = os.path.relpath(root, src_dir)
                dest_file_dir = os.path.join(dest_dir, relative_path)
                if not os.path.exists(dest_file_dir):
                    os.makedirs(dest_file_dir)
                dest_file = os.path.join(dest_file_dir, file)
                # Kopeeri fail sihtkausta, säilitades kataloogistruktuuri
                if not os.path.exists(dest_file):
                    shutil.copy2(src_file, dest_file)
                    
# Üksikute andmekogumite teekonnad
src_train_dirs = [
    'extracted_photos/chest_xray/chest_xray/train',
    'extracted_photos/Data/train',
    'extracted_photos/Train',
    'extracted_photos/chest_xray/train'
]

src_test_dirs = [
    'extracted_photos/chest_xray/chest_xray/test',
    'extracted_photos/Data/test',
    'extracted_photos/Test',
    'extracted_photos/chest_xray/test'
]

# Kombineeritud andmekogumi teekond
combined_train_dir = 'combined_dataset/train'
combined_test_dir = 'combined_dataset/test'

# Loome sümbollingid treening- ja testandmete jaoks
create_symlinks(src_train_dirs, combined_train_dir)
create_symlinks(src_test_dirs, combined_test_dir)

# ImageDataGenerators treeninguks ja testimiseks
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Loome treening- ja testimisgeneraatorid kombineeritud kataloogist
train_generator = train_datagen.flow_from_directory(
    combined_train_dir,
    target_size=(150, 150),
    batch_size=128,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    combined_test_dir,
    target_size=(150, 150),
    batch_size=128,
    class_mode='categorical'
)


Found 5216 images belonging to 3 classes.
Found 624 images belonging to 3 classes.


In [ ]:
# Laeme eelttreenitud VGG16 mudel ja lisame kohandatud kihid
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Külmutame baasmudeli kihid
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Treenime mudelit
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

# Hindame mudelit
test_loss, test_acc = model.evaluate(test_generator)
print(f"Testi täpsus: {test_acc}")

# Vabastame mõned baasmudeli kihid peenhäälestamiseks
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

# Hindame peenhäälestatud mudelit
test_loss, test_acc = model.evaluate(test_generator)
print(f"Peenhäälestatud testi täpsus: {test_acc}")


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,763,779 (60.13 MB)

 Trainable params: 1,049,091 (4.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

...

40/40 ━━━━━━━━━━━━━━━━━━━━ 195s 5s/step - accuracy: 0.7039 - loss: 0.6713 - val_accuracy: 0.8105 - val_loss: 0.5833
Epoch 4/5
 1/40 ━━━━━━━━━━━━━━━━━━━━ 2:01 3s/step - accuracy: 0.6979 - loss: 0.5873

2024-05-27 15:23:22.431002: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 103ms/step - accuracy: 0.6979 - loss: 0.5873 - val_accuracy: 0.8750 - val_loss: 0.5129
Epoch 5/5


2024-05-27 15:23:26.448356: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


40/40 ━━━━━━━━━━━━━━━━━━━━ 198s 5s/step - accuracy: 0.7114 - loss: 0.6509 - val_accuracy: 0.7285 - val_loss: 0.6427
5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.7159 - loss: 0.6432
Test accuracy: 0.7227563858032227
Epoch 1/50
27/40 ━━━━━━━━━━━━━━━━━━━━ 1:06 5s/step - accuracy: 0.6626 - loss: 0.7609

In [ ]:
# Funktsioon pildi laadimiseks ja eeltöötluseks
def load_and_preprocess_image(img_path, target_size=(150, 150)):
    img = load_img(img_path, target_size=target_size)  # Lae pilt ja muuda selle suurus
    img_array = img_to_array(img)  # Muuda pilt NumPy massiiviks
    img_array = np.expand_dims(img_array, axis=0)  # Lisa uus telg, et luua 4D tensor
    img_array /= 255.0  # Muuda väärtused vahemikku 0-1
    return img_array

# Funktsioon ennustuste tegemiseks uute piltide puhul
def predict_image(img_path):
    img_array = load_and_preprocess_image(img_path)  # Laadi ja eeltööta pilt
    prediction = model.predict(img_array)  # Tee ennustus mudeli abil
    return prediction


In [ ]:
# Näide uute piltide ennustamiseks

new_images = [
    "combined_dataset/test/TUBERCULOSIS/Tuberculosis-1.png",
    "combined_dataset/test/PNEUMONIA/person1_virus_6.jpeg",
    "combined_dataset/test/NORMAL/IM-0001-0001.jpeg",
    "combined_dataset/test/COVID19/COVID19(461).jpg"  # Asenda need oma uute piltide teedega
]

# Lisame kaustast "random Lung pics" juhuslikud pildid
new_images = new_images + ["random Lung pics/" + a for a in os.listdir("random Lung pics")]
print(new_images)

for img_path in new_images:
    img = Image.open(img_path)

    prediction = predict_image(img_path)
    class_index = np.argmax(prediction[0])
    class_label = list(train_generator.class_indices.keys())[class_index]
    print(f"Pilt {img_path} on ennustatud olevat {class_label} tõenäosusega {prediction[0][class_index]:.4f}")

    plt.imshow(img)
    plt.axis('off')
    display(plt.gcf())
